In [1]:
import csv
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# read xlsx file
def read_xlsx(file):
    df = pd.read_excel(file, header=0)
    return df

In [3]:
votes_meps_or = read_xlsx('data/EP9_RCVs_2022_06_22.xlsx')
voted_docs_or = read_xlsx('data/EP9_Voted_docs.xlsx')

In [4]:
print(votes_meps_or.shape)
votes_meps_or.head()

(811, 13469)


,WebisteEpID,Fname,Lname,FullName,Activ,Country,Party,EPG,Start,End,...,13450,13451,13452,13453,13454,13455,13456,13457,13458,13459
0,197490,Magdalena,ADAMOWICZ,"ADAMOWICZ, Magdalena",yes,Poland,Independent,EPP,2019-07-02,2024-12-31,...,2,2,2,1,1,2,1,2,1,1
1,189525,Asim,ADEMOV,"ADEMOV, Asim",yes,Bulgaria,Citizens for European Development of Bulgaria,EPP,2019-07-02,2024-12-31,...,2,2,2,1,2,2,1,2,1,1
2,124831,Isabella,ADINOLFI,"ADINOLFI, Isabella",yes,Italy,Forza Italia,EPP,2019-07-02,2024-12-31,...,2,2,2,1,2,2,1,2,1,1
3,197826,Matteo,ADINOLFI,"ADINOLFI, Matteo",yes,Italy,Lega,IDG,2019-07-02,2024-12-31,...,2,2,1,2,2,1,1,3,2,1
4,197403,Alex,AGIUS SALIBA,"AGIUS SALIBA, Alex",yes,Malta,Partit Laburista,S&D,2019-07-02,2024-12-31,...,3,3,1,2,2,2,3,3,2,1


In [5]:
total_votes_meps = int(votes_meps_or.columns[-1])
total_votes_meps

13459

In [6]:
print(voted_docs_or.shape)
voted_docs_or.head()

(13459, 24)


,Vote ID,File,Order of vote,Date,Title,Procedure,Leg/Non-Leg/Bud,Type of Vote,Voting Rule,Rapporteur,...,Policy area,Subject,Final vote?,Am No.,Author,RCV,Vote,Yes,No,Abs
0,1,1,1,15.07.2019,Tuesday - request by the GUE/NGL group,NaN,Non,Proposal for a decision,s,0,...,Foreign & security policy,Procedural vote,1.0,NaN,0,RCV,-,83,142,72
1,2,2,1,18.07.2019,Situation at the USA-Mexican border,NaN,Non,Joint motion for a resolution,s,0,...,Foreign & security policy,Paragraph 13,0.0,NaN,original text,RCV,+,311,269,33
2,3,2,2,18.07.2019,Situation at the USA-Mexican border,NaN,Non,Joint motion for a resolution,s,0,...,Foreign & security policy,Paragraph 14,0.0,NaN,original text,RCV,+,437,155,33
3,4,2,3,18.07.2019,Situation at the USA-Mexican border,NaN,Non,Joint motion for a resolution,s,0,...,Foreign & security policy,Paragraph 17,0.0,NaN,original text,RCV,+,441,158,27
4,5,2,4,18.07.2019,Situation at the USA-Mexican border,NaN,Non,Joint motion for a resolution,s,0,...,Foreign & security policy,Paragraph 19,0.0,NaN,original text,RCV,+,466,146,41


In [28]:
# Only choose legislative procedure with Parliament as co-legislator
voted_docs = voted_docs_or[voted_docs_or['Leg/Non-Leg/Bud'] == 'Leg']
voted_docs = voted_docs[voted_docs['Procedure'] != '*']
voted_docs = voted_docs[voted_docs['Procedure'] != '* ']

In [29]:
total_votes_leg = voted_docs.shape[0]
total_votes_leg

2582

How many times did MEPs on average not attend a vote?

In [9]:
# add column attendence with count of all 5s in each row
# get total votes by taking column header of last column

votes_meps_or['attendance'] = votes_meps_or.iloc[:, 10:].apply(lambda x: (x.ne(5).sum()/total_votes_meps), axis=1)


# average attendance
mean_att_EP9 = votes_meps_or['attendance'].mean()
print(f"Between 2019-2022, the average attendance of MEPs to all votes was {mean_att_EP9:.2%}")

Between 2019-2022, the average attendance of MEPs to all votes was 98.57%


How many votes were rejected?

In [30]:
# percentage of votes rejected
votes_rejected = voted_docs[voted_docs['Vote'] == '-'].shape[0]
votes_rejected_perc = votes_rejected/total_votes_leg
print(f"Between 2019-2022, {votes_rejected_perc:.2%} of votes were rejected")

Between 2019-2022, 38.26% of votes were rejected


How many laws were passed after the first reading?

In [37]:
# percentage of votes accepted or rejected after the first reading
votes_first_reading = voted_docs[voted_docs['Procedure'] == '***I'].shape[0]
votes_first_reading_perc = votes_first_reading/total_votes_leg
print(f"Between 2019-2022, {votes_first_reading_perc:.2%} of votes were accepted or rejected after the first reading.")

votes_first_reading = voted_docs[voted_docs['Procedure'] == '***II'].shape[0]
votes_first_reading_perc = votes_first_reading/total_votes_leg
print(f"Between 2019-2022, {votes_first_reading_perc:.2%} of votes were accepted or rejected after the second reading.")


votes_first_reading = voted_docs[voted_docs['Procedure'] == '***III'].shape[0]
votes_first_reading_perc = votes_first_reading/total_votes_leg
print(f"Between 2019-2022, {votes_first_reading_perc:.2%} of votes were accepted or rejected after the third reading.")


votes_first_reading = voted_docs[voted_docs['Procedure'] == '***'].shape[0]
votes_first_reading_perc = votes_first_reading/total_votes_leg
print(f"Between 2019-2022, {votes_first_reading_perc:.2%} of votes were on resolutions or motions of resolutions initiated by the parliament.")


Between 2019-2022, 92.53% of votes were accepted or rejected after the first reading.
Between 2019-2022, 4.73% of votes were accepted or rejected after the second reading.
Between 2019-2022, 0.00% of votes were accepted or rejected after the third reading.
Between 2019-2022, 2.71% of votes were on resolutions or motions of resolutions initiated by the parliament.


Establish Majority Labels

In [48]:
# function to calculate the percentage of votes for each group
def vote_perc(votes_meps, voted_docs, group):
    for col in votes_meps[10:]:
        if col["EPG"] == group:
            voted_docs[group] += 1
    return voted_docs.head()


In [67]:
# Reshape df_mps to calculate vote percentages
votes_meps = votes_meps_or.drop(columns=['Fname', 'Lname', 'WebisteEpID', "Activ", "Country", "Party", "Start", "End"])
df_mps_melted = votes_meps.melt(id_vars=['FullName', 'EPG'], var_name='Title', value_name='Vote')
#change vote to int
df_mps_melted['Vote'] = df_mps_melted['Vote'].astype(int)

print(df_mps_melted.shape)
display(df_mps_melted.head())

(10916060, 4)


,FullName,EPG,Title,Vote
0,"ADAMOWICZ, Magdalena",EPP,1,5
1,"ADEMOV, Asim",EPP,1,5
2,"ADINOLFI, Isabella",EPP,1,3
3,"ADINOLFI, Matteo",IDG,1,2
4,"AGIUS SALIBA, Alex",S&D,1,4


In [69]:
# Calculate the percentage of 'Yes' votes for each party on each law
def calculate_vote_percentage(df, party):
    party_df = df[df['EPG'] == party]
    return (party_df['Vote'] == 1).mean()

# Group by law and party and calculate percentage
party_votes = df_mps_melted.groupby(['Title', 'EPG']).apply(lambda x: calculate_vote_percentage(x, x['EPG'].iloc[0]))
party_votes = party_votes.unstack()  # This makes parties columns

# Merge the voting percentages back to df_laws
df_laws = voted_docs.merge(party_votes, left_on='Title', right_index=True, how='left')

# Create new columns based on conditions
df_laws['LM'] = (df_laws['S&D'] >= 0.80) & (df_laws['EPP'] == 0)
df_laws['RM'] = (df_laws['EPP'] >= 0.80) & (df_laws['S&D'] == 0)
df_laws['C'] = (df_laws['S&D'] >= 0.70) & (df_laws['EPP'] >= 0.70)

       Vote ID  File  Order of vote        Date  \
82          83    10              1  22.10.2019   
83          84    10              1  22.10.2019   
84          85    10              1  22.10.2019   
85          86    10              1  22.10.2019   
149        150    12              1  24.10.2019   
...        ...   ...            ...         ...   
13402    13403   134             39  08.06.2022   
13403    13404   134             40  08.06.2022   
13404    13405   134             41  08.06.2022   
13405    13406   134             42  08.06.2022   
13426    13427   135              1  09.06.2022   

                                                   Title Procedure  \
82     European Globalisation Adjustment Fund (2014-2...      ***I   
83     Fishing authorisations for Union fishing vesse...      ***I   
84     Implementation and financing of the EU general...       ***   
85     Periods of application of Regulation (EU) 2019...      ***I   
149    Financial assistance to Member

In [70]:
display(df_laws.head())

,Vote ID,File,Order of vote,Date,Title,Procedure,Leg/Non-Leg/Bud,Type of Vote,Voting Rule,Rapporteur,...,EPP,Greens/EFA,IDG,NI,REG,S&D,The Left,LM,RM,C
82,83,10,1,22.10.2019,European Globalisation Adjustment Fund (2014-2...,***I,Leg,Draft legislative resolution,s,Blinkevičiūtė,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
83,84,10,1,22.10.2019,Fishing authorisations for Union fishing vesse...,***I,Leg,Draft legislative resolution,s,Davies,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
84,85,10,1,22.10.2019,Implementation and financing of the EU general...,***,Leg,Draft legislative resolution,s,Van Overtveldt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
85,86,10,1,22.10.2019,Periods of application of Regulation (EU) 2019...,***I,Leg,Draft legislative resolution,s,Delli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
149,150,12,1,24.10.2019,Financial assistance to Member States to cover...,***I,Leg,Draft legislative resolution,s,Omarjee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False


In [44]:
def majority_calc(voted_docs):
    for item in voted_docs.iterrows():
        total_votes_cast = sum(int(voted_docs[item, "Yes"]), 
                               int(voted_docs[item, "No"]), 
                               int(voted_docs[item, "Abs"])
                               )
    if (voted_docs["S&D"].sum() > 0.8*total_votes_cast
        ) and (voted_docs["EPP"].sum() <
            0.2*total_votes_cast):
        voted_docs["LM"] = 1
        voted_docs["RM"] = 0
        voted_docs["C"] = 0
    elif (voted_docs["EPP"].sum() > 0.8*total_votes_cast
          ) and (voted_docs["S&D"].sum() < 0.2*total_votes_cast
                 ):
        voted_docs["RM"] = 1
        voted_docs["LM"] = 0
        voted_docs["C"] = 0
    elif (voted_docs["S&D"].sum() > 0.8*total_votes_cast
          ) and (voted_docs["EPP"].sum() > 0.8*total_votes_cast
                 ):
        voted_docs["C"] = 1
        voted_docs["LM"] = 0
        voted_docs["RM"] = 0
    return voted_docs.head()

In [55]:
# Sample data setup for df_mps
data_mps = {
    'MP': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank'],
    'Party': ['Party1', 'Party1', 'Party2', 'Party2', 'Party1', 'Party2'],
    'Law1': ['Yes', 'No', 'Yes', 'Yes', 'Yes', 'No'],
    'Law2': ['No', 'Yes', 'No', 'Yes', 'No', 'Yes']
}
df_mps = pd.DataFrame(data_mps)

display(df_mps)

# Sample data setup for df_laws
data_laws = {
    'Law': ['Law1', 'Law2'],
    'Description': ['Description of Law 1', 'Description of Law 2']
}
df_laws = pd.DataFrame(data_laws)

display(df_laws)

# Reshape df_mps to calculate vote percentages
df_mps_melted = df_mps.melt(id_vars=['MP', 'Party'], var_name='Law', value_name='Vote')

display(df_mps_melted.head())


,MP,Party,Law1,Law2
0,Alice,Party1,Yes,No
1,Bob,Party1,No,Yes
2,Charlie,Party2,Yes,No
3,David,Party2,Yes,Yes
4,Eve,Party1,Yes,No
5,Frank,Party2,No,Yes


,Law,Description
0,Law1,Description of Law 1
1,Law2,Description of Law 2


,MP,Party,Law,Vote
0,Alice,Party1,Law1,Yes
1,Bob,Party1,Law1,No
2,Charlie,Party2,Law1,Yes
3,David,Party2,Law1,Yes
4,Eve,Party1,Law1,Yes
5,Frank,Party2,Law1,No
6,Alice,Party1,Law2,No
7,Bob,Party1,Law2,Yes
8,Charlie,Party2,Law2,No
9,David,Party2,Law2,Yes
